In [4]:
#reddit account
#username:IES_Python
#email:95346725@fsv.cuni.cz

In [5]:
#pip install praw
import praw
import re
import time
import json
from praw.models import MoreComments
import pandas as pd
from datetime import date
credentials = {
    "client_id": "Mz-eGRSHPkpkaQ",
    "client_secret": "hNHgiDETfcYvbUweNZrSV-dAHt_mbw",
    "user_agent": "IES_python_stocks",
    "username": "IES_Python",
    "password": "IES_python"
}

In [6]:


class Reddit_Extractor:
    def __init__(self,credentials):
        self.reddit = self.redditApi()
        
    def redditApi(self):
        reddit = praw.Reddit(
            client_id = credentials['client_id'],
            client_secret = credentials['client_secret'],
            user_agent = credentials['user_agent'],
            username = credentials['username'],
            password = credentials['password'])
        return reddit
    
    def subredditData(self,subreddit,limit = 10):
        postList = self.reddit.subreddit(subreddit).hot(limit=limit)
        return postList     
        
    def showcasePrint(self,postList):
        for post in postList:
            print(post.title)
            print(post.distinguished)
            print(post.link_flair_text)
            '''for top_level_comment in submission.comments:
                print(i)
                print(top_level_comment.link_id)   
                print(top_level_comment.body)      
                print(top_level_comment.created_utc)
                i+=1'''    
            
    def subredditDF(self,postList):
        df = pd.DataFrame({'post_id': pd.Series([], dtype='str'),
                        'comment_id': pd.Series([], dtype='str'),
                        'subreddit': pd.Series([], dtype='str'),
                        'text_type': pd.Series([], dtype='str'),
                        'epoch_time': pd.Series([], dtype='int'),
                        'text': pd.Series([], dtype='str'),
                        'tickers': pd.Series([], dtype='str')})  
        for post in postList:
            #post extraction
            post_row = pd.DataFrame({'post_id': pd.Series([post.id], dtype='str'),
                                    'comment_id': pd.Series([], dtype='str'),
                                    'subreddit': pd.Series([post.subreddit.display_name], dtype='str'),
                                    'text_type': pd.Series(['post'], dtype='str'),
                                    'epoch_time': pd.Series([post.created_utc], dtype='int'),
                                    'text': pd.Series([post.title+post.selftext], dtype='str'),
                                   'tickers': pd.Series([], dtype='str')})   
            df = df.append(post_row) 
            #comments extraction
            submission = self.reddit.submission(id=post.id)
            submission.comment_sort = 'best'
            submission.comment_limit = 5
            for top_level_comment in submission.comments:
                if isinstance(top_level_comment, praw.models.MoreComments):
                    continue                    
                new_row = pd.DataFrame({'post_id': pd.Series([post.id], dtype='str'),
                                        'comment_id': pd.Series([top_level_comment.id], dtype='str'),
                                        'subreddit': pd.Series([top_level_comment.subreddit.display_name], dtype='str'),
                                        'text_type': pd.Series(['comment'], dtype='str'),
                                        'epoch_time': pd.Series([top_level_comment.created_utc], dtype='int'),
                                        'text': pd.Series([top_level_comment.body], dtype='str'),
                                       'tickers': pd.Series([], dtype='str')})  
                df = df.append(new_row)  
                                         
        return df
    def joinData(self,subreddits,limit):
        main_df = pd.DataFrame({'post_id': pd.Series([], dtype='str'),
                'comment_id': pd.Series([], dtype='str'),
                'subreddit': pd.Series([], dtype='str'),
                'text_type': pd.Series([], dtype='str'),
                'epoch_time': pd.Series([], dtype='int'),
                'text': pd.Series([], dtype='str'),
                'tickers': pd.Series([], dtype='str')})
        for subreddit in subreddits:
            posts_test = self.subredditData(subreddit = subreddit,limit = limit)
            start = time.time()
            df = self.subredditDF(posts_test)
            end = time.time()
            print("Elapsed time for subreddit",subreddit)
            print(end-start)
            main_df = main_df.append(df)
        return main_df

In [7]:
class TickerExtractor:
    def __init__(self):
        self.stockDict = self.openDict()
    def openDict(self):
        with open('stockDict.json') as json_file: 
            stockDict = json.load(json_file) #
        return stockDict
    def findNewTicker(self, postText,tickerDict): #method to expand possible dictionaries of stock tickers
        pass
    
    def tickerCounter(self, postText): #append number of mentions to dataframe
        #pattern  = r"(?:\$)?[A-Z][A-Z\d]+"
        mentionedTickers = []
        for key in self.stockDict:
            if re.search(self.stockDict[key],postText,re.IGNORECASE) is not None:
                mentionedTickers.append(key)
        mentionedTickers = list(set(mentionedTickers)) #unique values
        if len(mentionedTickers)>0:
            return mentionedTickers
        else:
            return None

In [8]:
#load posts
subreddits = ["stocks","investing","StockMarket","wallstreetbets"]
extr = Reddit_Extractor(credentials)
main_df = extr.joinData(subreddits = subreddits,limit = 100)

Elapsed time for subreddit stocks
31.445240259170532
Elapsed time for subreddit investing
26.87572741508484
Elapsed time for subreddit StockMarket
31.168322563171387
Elapsed time for subreddit wallstreetbets
70.7498471736908


In [9]:
tickerExtr = TickerExtractor()
main_df['tickers'] = main_df['text'].apply(tickerExtr.tickerCounter)

In [10]:
#observe dataframe
#main_df.shape
main_df.tickers[main_df.tickers.notna()].tolist()
main_df['subreddit'].value_counts()
#main_df.columns
#main_df.loc[main_df['text_type'] == 'comment']
#main_df.head()
#main_df.loc[0]['text']

wallstreetbets    449
stocks            439
investing         355
StockMarket       319
Name: subreddit, dtype: int64

In [11]:
#store object
today = date.today()
stringEpoch = today.strftime("%d_%m_%y")
#stringEpoch = stringEpoch + "_test"
s = ''
nameVars = ["D:\\School\\bartusek_cala_python\\daily_reddit_data\\",stringEpoch,".csv"]
main_df.to_csv(path_or_buf=s.join(nameVars),index=False,encoding='utf-8-sig')